# AiiDAlab QauntumESPRESSO App



In [ ]:
import re
from datetime import datetime, timedelta

import ipywidgets as ipw
import pandas as pd
from aiida import load_profile
from aiida.tools.query.calculation import CalculationQueryBuilder

load_profile()


process_label = "QeAppWorkChain"
projections = ["pk", "ctime", "state", "label"]


def query_data():
    builder = CalculationQueryBuilder()
    filters = builder.get_filters(
        process_label=process_label,
    )
    query_set = builder.get_query_set(
        filters=filters,
        order_by={"ctime": "desc"},
    )
    projected = builder.get_projected(
        query_set,
        projections=projections,
    )
    return projected


def parse_relative_time(time_str):
    """Custom parsing function for 'ctime' column"""
    # Get the current time
    now = datetime.now()
    # Match the number and the unit (h, D, M, m, S) from the string
    match = re.match(r"(\d+)([hDmMS]) ago", time_str)
    if match:
        value, unit = match.groups()
        value = int(value)
        
        # Subtract the respective time delta from the current time
        if unit == 'h':
            return now - timedelta(hours=value)
        elif unit == 'D':
            return now - timedelta(days=value)
        elif unit == 'M':
            # Assuming 1M = 30 days for simplicity
            return now - timedelta(days=value*30)
        elif unit == 'm':
            return now - timedelta(minutes=value)
        elif unit == 'S':
            return now - timedelta(seconds=value)
    # Return the original string if no match was found
    return time_str

In [ ]:
table = ipw.HTML()

css_style = """
        <style>
            .df { border: none; }
            .df tbody tr:nth-child(odd) { background-color: #e5e7e9; }
            .df tbody tr:nth-child(odd):hover { background-color:   #f5b7b1; }
            .df tbody tr:nth-child(even):hover { background-color:  #f5b7b1; }
            .df tbody td { min-width: 80px; text-align: center; border: none }
            .df th { text-align: center; border: none;  border-bottom: 1px solid black;}
        </style>
        """

df = pd.DataFrame(query_data()[1:], columns=projections)

unique_properties = set()
for index, row in df.iterrows():
    pk = row['pk']
    df.at[index, "ctime"] = parse_relative_time(row['ctime'])
    df.at[index, "formula"] = row['label'].split("structure")[-0]
    df.at[index, "relaxed"] = "is relaxed" in row['label']
    label_data = row['label'].split('properties on')
    if len(label_data) > 1:
        df.at[index, "properties"] = label_data[-1]
        properties = label_data[-1].split(',')
        # Extract unique properties
        unique_properties.update(properties)
    else:
        df.at[index, "properties"] = ""
    # add a column to the dataframe with a button to delete the archive
    df.at[index, 'Delete'] = f"""<a href="./delete.ipynb?pk={pk}" target="_blank">Delete</a>"""
    df.at[index, 'Inspect'] = f"""<a href="./qe.ipynb?pk={pk}" target="_blank">Inspect</a>"""
unique_properties = list(unique_properties)

# Create checkboxes for each unique property
property_checkboxes = [ipw.Checkbox(value=False, description=prop, Layout=ipw.Layout(description_width='initial'), indent=False) for prop in unique_properties]
properties_box = ipw.HBox(children=property_checkboxes, description='Properties:')
# Dropdown for job state
job_state_dropdown = ipw.Dropdown(
    options=['Finished', 'Except', 'Killed'],
    value='Finished',
    description='Job State:',
)
# Time range picker
time_start = ipw.DatePicker(description='Start Time:')
time_end = ipw.DatePicker(description='End Time:')
time_box = ipw.HBox([time_start, time_end])
# Button to apply filters
apply_filters_btn = ipw.Button(description='Apply Filters')

# Layout
filters_layout = ipw.VBox([ipw.HTML("<h2>Search results:</h2>"),
                           ipw.VBox([
                               ipw.HBox([ipw.HTML("<h>Properties: </h>"), properties_box]),
                               job_state_dropdown,
                               time_box,
                               apply_filters_btn])
])


def apply_filters(b):
    # Convert checkbox states to a list of selected properties
    selected_properties = [cb.description for cb in property_checkboxes if cb.value]
    filtered_df = df.copy()
    # Filter by job state
    filtered_df = filtered_df[filtered_df['state'].str.contains(job_state_dropdown.value)]
    # Filter by properties
    if selected_properties:
        filtered_df = filtered_df[filtered_df['properties'].apply(lambda x: all(prop in x for prop in selected_properties))]
    
    # Filter by time range
    if time_start.value and time_end.value:
        start_time = pd.to_datetime(time_start.value)
        end_time = pd.to_datetime(time_end.value)
        filtered_df = filtered_df[(filtered_df['ctime'] >= start_time) & (filtered_df['ctime'] <= end_time)]
    
    # Update table
    table.value = css_style + filtered_df.to_html(classes="df", escape=False, index=False)


apply_filters_btn.on_click(apply_filters)

filters_layout

In [ ]:
# delete the label column
df = df.drop(columns=['label'])
table.value = css_style + df.to_html(classes="df", escape=False, index=False)
table